In [5]:
import helpers
from selenium.webdriver import Remote, ChromeOptions
from bs4 import BeautifulSoup
import time
from datetime import datetime, timezone

options = ChromeOptions()
max_pages = 10
today = "2024-02-13"

now = datetime.now(timezone.utc)
today = now.strftime("%Y-%m-%d")

url_pattern = "https://news.ycombinator.com/front?day={day}&p={page}"
detail_pattern = "https://news.ycombinator.com/item?id={item_id}"
sbr_connection = helpers.get_sbr_connection()

In [6]:
# for page in range(1, 11):
#     print(page)

In [1]:
html_datas = []
with Remote(sbr_connection, options=options) as driver:
    for page in range(1, max_pages + 1):
        url = url_pattern.format(day=today, page=page)
        print(page, url)
        driver.get(url) # HTTP GET
        time.sleep(2)
        html_source = driver.page_source
        """
        Review the notebook
        7 - Scrape and Save Data to Local Files.ipynb
        to save files
        # with open('some-file.html', 'r') as f:
        #     f.write(html_source)
        """
        html_datas.append(html_source)

SyntaxError: unterminated string literal (detected at line 9) (3642852238.py, line 9)

In [8]:
post_data = []
for html_source in html_datas:
    soup = BeautifulSoup(html_source, 'html.parser')
    rows = soup.find_all('tr', class_="athing")
    for tr in rows:
        id = tr.attrs.get('id')
        next_tr = tr.find_next('tr')
        score_span = next_tr.find("span", class_="score")
        score = None
        if score_span:
            score = "".join([x for x in score_span.get_text() if x.isdigit()])
        title_element = tr.find("span", class_="titleline")
        text = title_element.get_text()
        target_links = [x.get('href') for x in tr.find_all('a') if x.get('href').startswith("http")]
        target_link_data = target_links[0] if len(target_links) == 1 else target_links
        detail_link = detail_pattern.format(item_id = id)
        post_data.append(
            {
                "id": id,
                "text": text,
                "target_link_data": target_link_data,
                "score": score,
                "thead_link": detail_link
            }
        )

In [10]:
len(post_data)

207